jusqu'a present, dans ce chapitre, nous nous sommes préoccupés de la réorganisation des données. Le filtrage, le nettoyage ainsi que d'autres transformation constituent une catégorie différente d'opération importantes.

## Supprimer les doublons
un DataFrame peut afficher des lignes dupliquées pour un certain nombre de raison. En voici un exemple :

In [17]:
import pandas as pd
import numpy as np
data = pd.DataFrame({'k1': ['un', 'deux'] * 3 + ['deux'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,un,1
1,deux,1
2,un,2
3,deux,3
4,un,3
5,deux,4
6,deux,4


La méthode duplicated de l'objet DataFrame renvoie une série booléenne indiquant si chaque ligne est un doublon(si elle a été observée dans une ligne précédente) ou non :

In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

de même, drop_duplicates renvoies un DataFrame correspondant aux cas dans lequels le tebleau duplicated est False :

In [19]:
data.drop_duplicates()

,k1,k2
0,un,1
1,deux,1
2,un,2
3,deux,3
4,un,3
5,deux,4


ces eux méthodes traitent par defaut toutes les colonnes. Mais on peux également spécifier n'importe quel sous-ensemble de celles-ci pour détecter les doublons. Supposons que nous ayons une colonne de valeur supplémentaire et qu'on veut filtrer les doublons uniquement sur la base de la colonne 'k1' 

In [20]:
data['v1'] = range(7)
data 

,k1,k2,v1
0,un,1,0
1,deux,1,1
2,un,2,2
3,deux,3,3
4,un,3,4
5,deux,4,5
6,deux,4,6


In [21]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,un,1,0
1,deux,1,1


duplicated et drop_duplicates conservent par defaut la premiere combinaison de valeurs observées. Passer keep= 'last' renverra la dernière :

In [22]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,un,1,0
1,deux,1,1
2,un,2,2
3,deux,3,3
4,un,3,4
6,deux,4,6


### Transformer des données à l'aide d'un fonction ou d'un mappage 
pour de nombreux jeux de données, on peut effectuer une transformation basé sur les valeur d'un tableau, d'une série ou d'une colonne dans un DataFrame. Considérons par exemple les données hypothétiques suivantes, collectées à partir de différents types de viandes : 

In [23]:
data = pd.DataFrame({'nourriture': ['bacon', 'porc braisé', 'bacon', 'Pastrami', 'corned beef', 'bacon', 'pastramie', 'jambon', 'saumon fumé'],
                     'gr': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,nourriture,gr
0,bacon,4.0
1,porc braisé,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastramie,3.0
7,jambon,5.0
8,saumon fumé,6.0


on suppose qu'on veut ajouter une colonne indiquant le genre d'animal dont provient chaque aliment. Ecrire uen correspondance entre chaque type de viande distinc et l'animal correspondant :

In [24]:
nourr_animal = {'bacon': 'porc',
                'porc braisé':'porc',
                'Pastrami': 'boeuf',
                'corned beef': 'boeuf',
                'jambon': 'porc',
                'saumon fumé': 'saumon',}

La méthode map d'un objet Series accepte une fonction ou un objet de type dict contenant un mappage, mais ici nous avons un petit problème : certains nom de viandes comprennent des majuscules et d'autres non. on doit convertir chaque valeur en minuscule avec la méthode de Series str.lower() :

In [25]:
lowercase = data['nourriture'].str.lower()
lowercase

0          bacon
1    porc braisé
2          bacon
3       pastrami
4    corned beef
5          bacon
6      pastramie
7         jambon
8    saumon fumé
Name: nourriture, dtype: object

In [26]:
data['animal'] = lowercase.map(nourr_animal)
data

,nourriture,gr,animal
0,bacon,4.0,porc
1,porc braisé,3.0,porc
2,bacon,12.0,porc
3,Pastrami,6.0,NaN
4,corned beef,7.5,boeuf
5,bacon,8.0,porc
6,pastramie,3.0,NaN
7,jambon,5.0,porc
8,saumon fumé,6.0,saumon


### Remplacer des valeurs
le remplissage des données manquantes avec la méthode fillna est un cas particulier de remplacement de valeur plsu général. comme on l'a deja vu map peut etre utilisé pour modifier un sous ensemble de valeur dans un objet, mais la méthode replace constituera un moyen plus simple et plus souple :

In [27]:
data = pd.Series([1, -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Les nombres -999 pourraient être des valeur sentinelles pour les données manquantes. Pour les remplacer par des valeur NA que pandas comprend, on peut utiliser replace, produisant une nouvelle série(a moin squ'on passe inplace= True) : 

In [28]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Si on veut remplacer plusieur valeur différente à la fois, on doit passer une liste puis la valeur de substitution  :

In [29]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Pour utiliser un remplacement différent pour chaque valeur, on passe une liste de substitus

In [30]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

l'argument passé peut aussi être un dict : 

In [31]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renommer les index d'un axe
Comme les valeur d'un objet Series, les étiquettes des axes peuvent être transformées de la mêmes façon par une fonction ou une certaine forme de mappage pour produire de nouveau objets étiquetés différemment. On peut modifier les axes en place sans créer de nouvelle structure de données. EXEMPLE : 